In [941]:
#import the library sqlite3
#YOUR CODE 
import sqlite3
import numpy as np
from sqlite3 import Error

#create a new database and open a database connection with the name example
#YOUR CODE 
conn = sqlite3.connect('db/exemple.db')

# create a data cursor cur in order execute SQL statements and fetch results from SQL queries
#YOUR CODE 
cur = conn.cursor()

# Create table with the column date, side, symbol, qty, price
#YOUR CODE 

try :
    rqc =   """CREATE TABLE IF NOT EXISTS test (
                id INTEGER NOT NULL,
                date DATETIME NOT NULL,
                side TEXT NOT NULL,
                symbol TEXT NOT NULL,
                qty INTEGER NOT NULL,
                price FLOAT NOT NULL,
                PRIMARY KEY (id)
            );"""
    cur.execute(rqc)
    conn.commit()
except sqlite3.Error as error :
    if (str(error) == "table test already exists") :
        try :
            rqd = "DELETE FROM test"
            cur.execute(rqd)
            conn.commit()
        except sqlite3.Error as error :
            print("ERROR:\ncouldn't reset table")
            print(error)
    else :
        print("ERROR:\ncouldn't create table")
        print(error)



In [942]:
# Insert 3 row of data: the stock APPLE ( AAPL) with the date of transaction, the nature of side ( BUY or SELL),
#the quantity of the stock traded and the price
#YOUR CODE 
rq_insert = """INSERT INTO test (date, side, symbol, qty, price) VALUES 
('2022-01-19', 'BUY', 'AAPL', 3, 79.47), 
('2022-10-04', 'SELL', 'AAPL', 7, 19.99), 
('2021-11-24', 'BUY', 'AAPL', 13, 46.01);"""

conn.execute("PRAGMA busy_timeout = 30000")
count = cur.execute(rq_insert)

# Save (commit) the changes
#YOUR CODE 
conn.commit()
print(count)


In [943]:
#return all resulting rows
#YOUR CODE 
request_rows = "SELECT * FROM test"
cur.execute(request_rows)
all_genres = cur.fetchall()
print(all_genres)

# Close the connection
#YOUR CODE 

conn.close()

[(1, '2022-01-19', 'BUY', 'AAPL', 3, 79.47), (2, '2022-10-04', 'SELL', 'AAPL', 7, 19.99), (3, '2021-11-24', 'BUY', 'AAPL', 13, 46.01), (4, '2022-01-19', 'BUY', 'AAPL', 3, 79.47), (5, '2022-10-04', 'SELL', 'AAPL', 7, 19.99), (6, '2021-11-24', 'BUY', 'AAPL', 13, 46.01), (7, '2022-01-19', 'BUY', 'AAPL', 3, 79.47), (8, '2022-10-04', 'SELL', 'AAPL', 7, 19.99), (9, '2021-11-24', 'BUY', 'AAPL', 13, 46.01), (10, '2022-01-19', 'BUY', 'AAPL', 3, 79.47), (11, '2022-10-04', 'SELL', 'AAPL', 7, 19.99), (12, '2021-11-24', 'BUY', 'AAPL', 13, 46.01), (13, '2022-01-19', 'BUY', 'AAPL', 3, 79.47), (14, '2022-10-04', 'SELL', 'AAPL', 7, 19.99), (15, '2021-11-24', 'BUY', 'AAPL', 13, 46.01), (16, '2022-01-19', 'BUY', 'AAPL', 3, 79.47), (17, '2022-10-04', 'SELL', 'AAPL', 7, 19.99), (18, '2021-11-24', 'BUY', 'AAPL', 13, 46.01), (19, '2022-01-19', 'BUY', 'AAPL', 3, 79.47), (20, '2022-10-04', 'SELL', 'AAPL', 7, 19.99), (21, '2021-11-24', 'BUY', 'AAPL', 13, 46.01), (22, '2022-01-19', 'BUY', 'AAPL', 3, 79.47), (23,

## the construction of a financial database

Let’s start simple and only consider the structure for the equity data that we are gathering for now – in our case the constituents of the S&P500 index. We can tag on the various tables and possibly whole databases to the mix as and when we need to, i.e. when we start to deal with the spot currency data and fund data that we plan to include.

the  plan is to segregate theequity data into 4 separate tables, using unique ids and foreign keys to create the relevant relationships and links between them.

The tables will be as follows:

<img src='C:/Users/User/Desktop/Formations_Data-LAPTOP-JP680GIQ/Formations_Data-LAPTOP-JP680GIQ/Organisme_formation/Ib_cegos/AXA/MCD.png'>

<img src='C:\Users\User\Desktop\Formations_Data-LAPTOP-JP680GIQ\Formations_Data-LAPTOP-JP680GIQ\Organisme_formation\Ib_cegos\AXA\MCD.png'>

In [944]:
#import the library os and the module Error of the library sqlite3
import os
import sqlite3
from sqlite3 import Error

#YOUR CODE
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    return conn
     


#YOUR CODE 
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
    
        

#the name of the database
db_name = "db/data.db"


def main():
    
    database = db_name

    
    # create the tables according to the MCD
    sql_create_exchange_table = """ CREATE TABLE IF NOT EXISTS exchange (
                                        id integer ,
                                        name text NOT NULL,
                                        currency text NOT NULL,
                                        code text NOT NULL UNIQUE,
                                        PRIMARY KEY(id)
                                    ); """
    
    sql_create_company_table = """  CREATE TABLE IF NOT EXISTS company (    
                                        id integer ,
                                        name text NOT NULL,
                                        industry text,
                                        sector text,
                                        hq_location text,
                                        security_id integer,
                                        PRIMARY KEY(id),
                                        FOREIGN KEY (security_id) REFERENCES security (id)
                                    ); """
    
    
    sql_create_security_table = """ CREATE TABLE IF NOT EXISTS security (
                                        id integer,
                                        ticker text NOT NULL,
                                        name text NOT NULL,
                                        company_id integer,
                                        exchange_id integer,
                                        PRIMARY KEY(id),
                                        FOREIGN KEY (company_id) REFERENCES company (id),
                                        FOREIGN KEY (exchange_id) REFERENCES exchange (id)
                                    ); """
    
    sql_create_security_price_table =  """ CREATE TABLE IF NOT EXISTS security_price (
                                        id integer,
                                        date text NOT NULL,
                                        open decimal NOT NULL,
                                        high decimal NOT NULL,
                                        low decimal NOT NULL,
                                        close decimal NOT NULL,
                                        volume integer,
                                        adj_close decimal NOT NULL,
                                        security_id integer,
                                        PRIMARY KEY(id),
                                        FOREIGN KEY (security_id) REFERENCES security (id)
                                    ); """

    
    # create a database connection    
    conn = create_connection(database)
    
    # create tables
    if conn is not None:
        # create exchange table
        create_table(conn, sql_create_exchange_table)
    
        # create company table
        create_table(conn, sql_create_company_table)    
        
        # create security table
        create_table(conn, sql_create_security_table)    
        
        # create security_price table
        create_table(conn, sql_create_security_price_table)
    
        
    else:
        print("Error! cannot create the database connection.")
main()

2.6.0


In [945]:
#import the library pandas , requests and io
import pandas as pd
exchange_data = pd.read_csv("https://www.iso20022.org/sites/default/files/ISO10383_MIC/ISO10383_MIC.csv", encoding='iso-8859-1' )


In [946]:
exchange_data.head(30)

,MIC,OPERATING MIC,OPRT/SGMT,MARKET NAME-INSTITUTION DESCRIPTION,LEGAL ENTITY NAME,LEI,MARKET CATEGORY CODE,ACRONYM,ISO COUNTRY CODE (ISO 3166),CITY,WEBSITE,STATUS,CREATION DATE,LAST UPDATE DATE,LAST VALIDATION DATE,EXPIRY DATE,COMMENTS
0,DRSP,DRSP,OPRT,EURONEXT UK - REPORTING SERVICES,EURONEXT LONDON LIMITED,969500HMVSZ0TCV65D58,APPA,NaN,GB,LONDON,WWW.EURONEXT.COM,ACTIVE,20210927,20210927,20210927.0,NaN,APPROVED PUBLICATION ARRANGEMENT.
1,XCNQ,XCNQ,OPRT,CANADIAN SECURITIES EXCHANGE,"CNSX MARKETS, INC.",NaN,RMKT,CSE LISTED,CA,TORONTO,WWW.THECSE.COM,ACTIVE,20090427,20210927,20210927.0,NaN,FORMERLY KNOWN AS PURE (CSE OTHER LISTED) FORM...
2,PURE,XCNQ,SGMT,CANADIAN SECURITIES EXCHANGE - PURE,"CNSX MARKETS, INC.",NaN,NSPD,CSE-PURE,CA,TORONTO,WWW.THECSE.COM,ACTIVE,20061225,20210927,20210927.0,NaN,FORMERLY KNOWN AS PURE (CSE OTHER LISTED) FORM...
3,ZODM,ZODM,OPRT,ZODIA MARKETS,ZODIA MARKETS HOLDINGS LIMITED,5493006GZOHISU7JTC27,NSPD,NaN,GB,LONDON,WWW.ZODIA-MARKETS.COM,ACTIVE,20210927,20210927,20210927.0,NaN,CRYPTOASSET BROKERAGE AND EXCHANGE.
4,FNFX,BGCF,SGMT,FENICS FX ECN,"FENICS FX, LLC",549300I0NONVB9S5DZ56,NSPD,NaN,US,NEW YORK,WWW.FENICSFX.COM,ACTIVE,20210927,20210927,20210927.0,NaN,ELECTRONIC COMMUNICATIONS NETWORK.
5,NORX,NORX,OPRT,NASDAQ OSLO ASA,NASDAQ OSLO ASA,5493003IZZWOHHJY1L88,RMKT,NaN,NO,OSLO,WWW.NASDAQ.COM/SOLUTIONS/EUROPEAN-COMMODITIES,ACTIVE,20110523,20210927,20210927.0,NaN,NaN
6,POSE,POSE,OPRT,PORTFOLIO STOCK EXCHANGE,"EUROPEAN DIGITAL SECURITIES EXCHANGE, S.L.",959800UP9ANDBHTKJ408,MLTF,NaN,ES,MADRID,WWW.PORTFOLIO.EXCHANGE,ACTIVE,20210927,20210927,NaN,NaN,MULTILATERAL TRADING FACILITY.
7,PUND,PUND,OPRT,PUNDION LLC,PUNDION LLC,549300FOYST42PEQ1R51,NSPD,NaN,US,NEW YORK,WWW.PUNDION.COM,ACTIVE,20210927,20210927,20210927.0,NaN,EQUITY PRODUCTS.
8,UCBG,UCBG,OPRT,UNICREDIT BULBANK AD,UNICREDIT BULBANK AD,549300Z7V2WOFIMUEK50,SINT,NaN,BG,SOFIA,WWW.UNICREDITBULBANK.BG,ACTIVE,20210927,20210927,20210927.0,NaN,SYSTEMATIC INTERNALISER.
9,NZFX,XASX,SGMT,ASX - NEW ZEALAND FUTURES & OPTIONS,AUSTRALIAN SECURITIES EXCHANGE (ASX),549300SDXT36XYPVWI79,NSPD,ASX,AU,SYDNEY,WWW.ASX.COM.AU,ACTIVE,20070924,20210927,20210927.0,NaN,NaN


In [947]:
exchange_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2561 entries, 0 to 2560
Data columns (total 17 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   MIC                                  2561 non-null   object 
 1   OPERATING MIC                        2561 non-null   object 
 2   OPRT/SGMT                            2561 non-null   object 
 3   MARKET NAME-INSTITUTION DESCRIPTION  2561 non-null   object 
 4   LEGAL ENTITY NAME                    495 non-null    object 
 5   LEI                                  1833 non-null   object 
 6   MARKET CATEGORY CODE                 2559 non-null   object 
 7   ACRONYM                              890 non-null    object 
 8   ISO COUNTRY CODE (ISO 3166)          2560 non-null   object 
 9   CITY                                 2559 non-null   object 
 10  WEBSITE                              2387 non-null   object 
 11  STATUS                        

In [948]:
exchange_data = exchange_data[['ISO COUNTRY CODE (ISO 3166)', 'MIC', 'MARKET NAME-INSTITUTION DESCRIPTION', 'ACRONYM']]
exchange_data.head()


,ISO COUNTRY CODE (ISO 3166),MIC,MARKET NAME-INSTITUTION DESCRIPTION,ACRONYM
0,GB,DRSP,EURONEXT UK - REPORTING SERVICES,NaN
1,CA,XCNQ,CANADIAN SECURITIES EXCHANGE,CSE LISTED
2,CA,PURE,CANADIAN SECURITIES EXCHANGE - PURE,CSE-PURE
3,GB,ZODM,ZODIA MARKETS,NaN
4,US,FNFX,FENICS FX ECN,NaN


In [949]:
exchange_data.rename(columns={'ISO COUNTRY CODE (ISO 3166)' : 'country_code', 
                              'MIC':'code', 
                              'MARKET NAME-INSTITUTION DESCRIPTION': 'name',
                              'ACRONYM':'acronym'}, inplace=True)
exchange_data.head()

,country_code,code,name,acronym
0,GB,DRSP,EURONEXT UK - REPORTING SERVICES,NaN
1,CA,XCNQ,CANADIAN SECURITIES EXCHANGE,CSE LISTED
2,CA,PURE,CANADIAN SECURITIES EXCHANGE - PURE,CSE-PURE
3,GB,ZODM,ZODIA MARKETS,NaN
4,US,FNFX,FENICS FX ECN,NaN


In [950]:
exchange_data['id'] = exchange_data.index
exchange_data.head(10)

,country_code,code,name,acronym,id
0,GB,DRSP,EURONEXT UK - REPORTING SERVICES,NaN,0
1,CA,XCNQ,CANADIAN SECURITIES EXCHANGE,CSE LISTED,1
2,CA,PURE,CANADIAN SECURITIES EXCHANGE - PURE,CSE-PURE,2
3,GB,ZODM,ZODIA MARKETS,NaN,3
4,US,FNFX,FENICS FX ECN,NaN,4
5,NO,NORX,NASDAQ OSLO ASA,NaN,5
6,ES,POSE,PORTFOLIO STOCK EXCHANGE,NaN,6
7,US,PUND,PUNDION LLC,NaN,7
8,BG,UCBG,UNICREDIT BULBANK AD,NaN,8
9,AU,NZFX,ASX - NEW ZEALAND FUTURES & OPTIONS,ASX,9


In [951]:
exchange_data['currency']=exchange_data.country_code.map({'US':'USD', 'GB':'GBP', 'DE':'EUR','CA':'CAD','NO':'EUR','ES':'EUR','BG':'EUR'})
exchange_data[['id', 'name', 'currency', 'code']].head(10)
exchange = exchange_data[['id', 'name', 'currency', 'code']]
# create the column currency in the the dataframe exhange_data whose values 
#are mapped with the country code : US => USD, GB =>GBP, DE =>EUR

In [952]:
# save the data in a sql table exchange in the data db_name
conn = create_connection("db/data.db")
cur = conn.cursor()
exchange.to_sql('exchange',conn,if_exists='replace',index=False)
conn.commit()
conn.close()

2.6.0


In [953]:
# scrape wiki table with symbols and details of Dow Jones constituents
dj_constituents = pd.read_html('https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average#Components')[1]
dj_constituents.rename(columns={'Company':'name', 'Industry': 'industry', 'Symbol': 'ticker'}, inplace=True)
dj_constituents.head()

,name,Exchange,ticker,industry,Date added,Notes,Index weighting
0,3M,NYSE,MMM,Conglomerate,1976-08-09,As Minnesota Mining and Manufacturing,2.88%
1,American Express,NYSE,AXP,Financial services,1982-08-30,NaN,3.56%
2,Amgen,NASDAQ,AMGN,Biopharmaceutical,2020-08-31,NaN,4.88%
3,Apple,NASDAQ,AAPL,Information technology,2015-03-19,NaN,3.15%
4,Boeing,NYSE,BA,Aerospace and defense,1987-03-12,NaN,3.40%


In [954]:
# scrape wiki table with symbols and details of s&P500 constituents
sp_constituents = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]#.to_csv('constintuents_current.csv', index=False)
sp_constituents.rename(columns={'Symbol': 'ticker', 'Security':'name', 'Headquarters Location': 'hq_location',
                                'GICS Sector': 'sector', 'GICS Sub-Industry':'industry'},inplace=True)
sp_constituents.head()

,ticker,name,SEC filings,sector,industry,hq_location,Date first added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
3,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981


In [955]:
#concetenate the previous tables/dataframe sp_constituents & dj_constituents with  the columns'name', 'industry', 'sector', 'hq_location'
company_table = pd.concat([sp_constituents, dj_constituents])
company_table = company_table[['name', 'industry', 'sector', 'hq_location']]
#drop duplicate
company_table = company_table.drop_duplicates()
#sort
company_table = company_table.sort_values('name')
#reset index
company_table = company_table.reset_index()
#create the colum id based on the index
company_table['id'] = company_table.index



In [956]:
company_table

,index,name,industry,sector,hq_location,id
0,0,3M,Industrial Conglomerates,Industrials,"Saint Paul, Minnesota",0
1,0,3M,Conglomerate,NaN,NaN,1
2,1,A. O. Smith,Building Products,Industrials,"Milwaukee, Wisconsin",2
3,7,ADM,Agricultural Products,Consumer Staples,"Chicago, Illinois",3
4,9,ADP,Data Processing & Outsourced Services,Information Technology,"Roseland, New Jersey",4
...,...,...,...,...,...,...
528,499,Zebra Technologies,Electronic Equipment & Instruments,Information Technology,"Lincolnshire, Illinois",528
529,500,Zimmer Biomet,Health Care Equipment,Health Care,"Warsaw, Indiana",529
530,501,Zions Bancorporation,Regional Banks,Financials,"Salt Lake City, Utah",530
531,502,Zoetis,Pharmaceuticals,Health Care,"Parsippany, New Jersey",531


In [957]:
#concetenate the previous tables/dataframe sp_constituents & dj_constituents with  the columns'ticker', 'name'
#drop duplicate
#sort
#reset index
#create the colum id based on the index

#YOUR CODE 
security_table = pd.concat([sp_constituents, dj_constituents])
security_table = security_table[['ticker', 'name']]
security_table = security_table.drop_duplicates()
security_table = security_table.sort_values('ticker')
security_table = security_table.reset_index()
security_table['id'] = security_table.index

In [958]:
security_table

,index,ticker,name,id
0,13,A,Agilent Technologies,0
1,30,AAL,American Airlines Group,1
2,10,AAP,Advance Auto Parts,2
3,3,AAPL,Apple,3
4,45,AAPL,Apple Inc.,4
...,...,...,...,...
509,498,YUM,Yum! Brands,509
510,500,ZBH,Zimmer Biomet,510
511,499,ZBRA,Zebra Technologies,511
512,501,ZION,Zions Bancorporation,512


In [959]:
#create the foreign key :  populating the security table with the company_id
#first step : create a dictionary with key, value pairs being the name of the company and the corresponding id 
#within its own table, which will be our foreign key in the second table.

#YOUT CODE 
company_merge = company_table.merge(security_table, on='name' , how = 'inner')
company_id_mapper = company_merge[['id_x', 'name', 'industry', 'sector', 'hq_location', 'id_y']]
company_id_mapper

,id_x,name,industry,sector,hq_location,id_y
0,0,3M,Industrial Conglomerates,Industrials,"Saint Paul, Minnesota",313
1,1,3M,Conglomerate,NaN,NaN,313
2,2,A. O. Smith,Building Products,Industrials,"Milwaukee, Wisconsin",40
3,3,ADM,Agricultural Products,Consumer Staples,"Chicago, Illinois",13
4,4,ADP,Data Processing & Outsourced Services,Information Technology,"Roseland, New Jersey",14
...,...,...,...,...,...,...
528,528,Zebra Technologies,Electronic Equipment & Instruments,Information Technology,"Lincolnshire, Illinois",511
529,529,Zimmer Biomet,Health Care Equipment,Health Care,"Warsaw, Indiana",510
530,530,Zions Bancorporation,Regional Banks,Financials,"Salt Lake City, Utah",512
531,531,Zoetis,Pharmaceuticals,Health Care,"Parsippany, New Jersey",513


In [960]:
#Seconde step: create the foreign key company id 
#create the foreign key : populate the company table with the security_id:
conn = create_connection("db/data.db")
cur = conn.cursor()
company_id_mapper.to_sql('company',conn,if_exists='replace',index=False)
conn.commit()
conn.close()

2.6.0


In [961]:
#Reda the nyse.csv and the nasdaq.csv
nyse = pd.read_csv("nyse.csv")
nasdaq = pd.read_csv("nasdaq.csv")

nyse.head()

,Symbol,Name,LastSale,MarketCap,IPOyear,Sector,industry,Summary Quote,Unnamed: 8
0,DDD,3D Systems Corporation,7.66,$927.89M,NaN,Technology,Computer Software: Prepackaged Software,https://old.nasdaq.com/symbol/ddd,NaN
1,MMM,3M Company,169.80,$97.81B,NaN,Health Care,Medical/Dental Instruments,https://old.nasdaq.com/symbol/mmm,NaN
2,WBAI,500.com Limited,2.95,$126.85M,2013.0,Consumer Services,Services-Misc. Amusement & Recreation,https://old.nasdaq.com/symbol/wbai,NaN
3,EGHT,8x8 Inc,16.75,$1.75B,NaN,Technology,EDP Services,https://old.nasdaq.com/symbol/eght,NaN
4,AHC,A.H. Belo Corporation,1.44,$34.39M,NaN,Consumer Services,Newspapers/Magazines,https://old.nasdaq.com/symbol/ahc,NaN


In [962]:
nasdaq = nasdaq.sort_values('Symbol')
nasdaq.head()

,Symbol,Name,LastSale,MarketCap,ADR TSO,IPOyear,Sector,Industry,Summary Quote,Unnamed: 9
336,AACG,ATA Creativity Global,1.230,1.456645e+07,11842642.0,NaN,Consumer Services,Other Consumer Services,https://old.nasdaq.com/symbol/aacg,NaN
312,AACQ,Artius Acquisition Inc.,9.710,8.793619e+08,NaN,2020.0,Finance,Business Services,https://old.nasdaq.com/symbol/aacq,NaN
313,AACQU,Artius Acquisition Inc.,10.090,0.000000e+00,NaN,2020.0,Finance,Business Services,https://old.nasdaq.com/symbol/aacqu,NaN
314,AACQW,Artius Acquisition Inc.,1.305,0.000000e+00,NaN,2020.0,Finance,Business Services,https://old.nasdaq.com/symbol/aacqw,NaN
201,AAL,"American Airlines Group, Inc.",12.600,6.407871e+09,NaN,NaN,Transportation,Air Freight/Delivery Services,https://old.nasdaq.com/symbol/aal,NaN


In [963]:
#populate the exchange_id column : assign the correct exchange_id for each stock in the security table:
# the exchange_id of the nyse stocks will be 1300
import numpy as np

security_merge = security_table.merge(company_id_mapper, on='name', how = 'inner')
security_table = security_merge[['id', 'ticker', 'name', 'id_x']]
nyse_merge = pd.merge(security_table, nyse['Symbol'], left_on='ticker', right_on='Symbol', how='left').merge(nasdaq['Symbol'], left_on='ticker', right_on='Symbol', how='left')
#nyse_merge = nyse_merge[['id', 'ticker', 'name', 'id_x', 'Symbol']]
nyse_merge['exchange_id'] = np.where((nyse_merge['Symbol_x'].notnull()), 1300, (np.where(nyse_merge['Symbol_y'].notnull(), 367, np.nan)))
security_table = nyse_merge[['id', 'ticker', 'id_x', 'exchange_id']] 

In [964]:
#populate the exchange_id column : assign the correct exchange_id for each stock in the security table:
# the exchange_id of  will be 1300 for the nyse stocks and 367 for the nasdaq stocks
#YOUR CODE

#YOUR CODE 
security_table = security_table[security_table['exchange_id'] != 'nan']
security_table.head(100)

,id,ticker,id_x,exchange_id
0,0,A,17,1300.0
1,1,AAL,33,367.0
2,2,AAP,15,1300.0
3,3,AAPL,49,367.0
4,4,AAPL,50,367.0
...,...,...,...,...
95,92,CF,90,1300.0
96,93,CFG,126,1300.0
97,94,CHD,119,1300.0
98,95,CHRW,87,367.0


In [965]:
#save the data in the sql table security and company of the database db_name
conn = create_connection("db/data.db")
cur = conn.cursor()
security_table.to_sql('security',conn,if_exists='replace',index=False)
conn.commit()
conn.close()

2.6.0


In [ ]:
#Extraction the prices of YAHOO: WARNING this action may take 15 min !!!
from tqdm import tqdm_notebook as tqdm
import pandas_datareader.data as web
stock_pricing_dfs = []
for stock_id in tqdm(security_table['id']):
    try:
        stock_pricing_df = web.DataReader(security_table.iloc[stock_id]['ticker'], 
                           start='2010-1-1', 
                           end='2020-08-31', 
                           data_source='yahoo')
        stock_pricing_df['security_id'] = stock_id
        stock_pricing_dfs.append(stock_pricing_df)
    except:
        pass
security_price_table = pd.concat(stock_pricing_dfs)

C:\Users\Administrateur\AppData\Local\Temp\ipykernel_6556\451006209.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for stock_id in tqdm(security_table['id']):


  0%|          | 0/533 [00:00<?, ?it/s]

In [ ]:
security_price_table.info()

In [ ]:
#insert data in the table security_price
conn = create_connection("db/data.db")
cur = conn.cursor()
security_price_table.columns = ['high', 'low', 'open', 'close', 'volume', 'adj_close', 'security_id']
security_price_table.reset_index(inplace=True)
security_price_table['id'] = security_price_table.index
print(security_price_table.head())
security_price_table.to_sql("security_price", conn, if_exists="append", index=False)
conn.commit()
conn.close()